In [1]:
import pandas as pd
import requests
import numpy
from tqdm import tqdm
import json

In [23]:
hostnames = pd.read_feather('./feather_hostnames_v2')
hostnames.fillna(value=numpy.nan, inplace=True)
count=0
for ind, row in tqdm(hostnames.iterrows(), total = hostnames.shape[0]):
    try:
        if not numpy.isnan(row['details']):
            continue
    except TypeError:
        continue
    try:
        r = requests.get(f"https://ipapi\.co/{row['ip_addr']}/json/")
        if r.status_code != 200:
            break
        hostnames.loc[ind, 'details'] = r.text
    except:
        print('Connection blocked')
        break
    count+=1
    if count==100:
        break



  0%|                                                                                       | 0/252430 [00:00<?, ?it/s]

Connection blocked


In [26]:
hostnames_copy = hostnames.copy()
hostnames = hostnames[~hostnames['details'].isnull()]

In [37]:
def get_det(val):
    val = json.loads(val)
    return val['city'], val['region_code'], val['country'], val['asn'], val['org']
    
hostnames['city'], hostnames['region_code'], hostnames['country'], hostnames['asn'], hostnames['org'] = zip(*hostnames[~hostnames['details'].isnull()]['details'].map(get_det))

In [60]:
hostnames = hostnames_copy.merge(hostnames, how='left')
hostnames.to_feather('./feather_hostnames_v2')